In [1]:
#!pip install psycopg2-binary
#!pip install pydub
#!pip install librosa
#!pip install wave
import librosa
import wave
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import base64
import yaml
import psycopg2
from psycopg2.extras import RealDictCursor
import matplotlib.pyplot as plt
import json
import scipy.stats as stats
import pandas as pd
# import statsmodels.api as sm
import os
import seaborn as sns
sns.set()
from joblib import dump
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC, SVC
from mpl_toolkits.mplot3d import Axes3D

## Connect to DB

In [2]:
file = open('/home/aaron/Trabajo/interaction-tracker-updated/PositionTracker/src/Data_Processing/config.yml', 'r')
cfg = yaml.load(file)
conn = psycopg2.connect(database=cfg['postgreSQL']['dbname'], user=cfg['postgreSQL']['user'],
                             password=cfg['postgreSQL']['pass'], host=cfg['postgreSQL']['host'],
                             port=cfg['postgreSQL']['port'])

cur = conn.cursor(cursor_factory=RealDictCursor)

## Create dataset

In [3]:
df = pd.DataFrame(columns=[['timestamp', 'recorder']+
                           ['mfcc_'+str(i) for i in range(1,21)]+
                           ['rms', 'spectral_centroid', 'zero_crossing_rate']])
df

,timestamp,recorder,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,...,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,rms,spectral_centroid,zero_crossing_rate


## Extract audio features

In [4]:
def extract_audio_features():
    y, sr = librosa.load("audio.wav", sr=32000)

    # Mel-frequency cepstral coefficients
    mfcc = librosa.feature.mfcc(y, sr, n_mfcc=20)

    #print(mfcc)

    # Root-Mean-square (RMS) : obtiene la presion de la señal de audio en diferentes puntos, la eleva al cuadrado, hace la media
    # y luego la raíz cuadrada
    rms = librosa.feature.rms(y)


    #print(rms)
    # Spectral-centroid: Indica donde está el "centro de masa" de la señal. Internamente, se calcula la FFT y se hace
    # una media ponderada de las frecuencias de la señal con sus magnitudes como ponderaciones.
    spec_centr = librosa.feature.spectral_centroid(y, sr)

    #print(spec_centr)

    # Zero-crossing rate: indica las veces en las que la señal ha pasado por el valor 0 (ha pasado de positivo-negativo
    # o de negativo-positivo)

    zero_cross = librosa.feature.zero_crossing_rate(y)

    return ([np.mean(mfcc[i]) for i in range(0,len(mfcc))], np.mean(rms), np.mean(spec_centr), np.mean(zero_cross))
    #print(zero_cross)

## Decode audio data

In [9]:
MOVILES = ['TARGETDEV-oz20sk', 'TARGETDEV-aw2zrw'] #tendrá los nombres de los 2 móviles
columns = df.columns


cur.execute("SELECT * FROM audiodata WHERE recorderdevice=%(recorderdevice)s ORDER BY time DESC", {'recorderdevice':MOVILES[1]})
audio_enc = cur.fetchall()

for i in range(0,len(audio_enc)):
    audio_dec = base64.b64decode(audio_enc[i]['audiodata'])

    wavfile = wave.open("audio.wav", 'wb')
    wavfile.setnchannels(1)
    wavfile.setsampwidth(1)
    wavfile.setframerate(32000)
    wavfile.writeframes(audio_dec)
    wavfile.close()

    # return the audio features for file "audio.wav"
    mfcc, rms, spec_cent, zero_cross = extract_audio_features()

    timestamp = audio_enc[i]['time']
    recorder = audio_enc[i]['recorderdevice']

    new_row = np.array([np.concatenate(([timestamp, recorder], [mfcc[i] for i in range(0,len(mfcc))], [rms, spec_cent, zero_cross]),axis=0)])
    df2 = pd.DataFrame(new_row, columns=columns)
    df = df.append(df2, ignore_index=True)

## Export dataset to .csv file

In [8]:
f = open('audioDataset.csv', 'w')
df.to_csv(f, sep=',')

## Analyse audio data

In [ ]:
y, sr = librosa.load("audio.wav", sr=32000)

# Mel-frequency cepstral coefficients
mfcc = librosa.feature.mfcc(y, sr, n_mfcc=20)

print(mfcc)

# Root-Mean-square (RMS) : obtiene la presion de la señal de audio en diferentes puntos, la eleva al cuadrado, hace la media
# y luego la raíz cuadrada
rms = librosa.feature.rms(y)


print(rms)
# Spectral-centroid: Indica donde está el "centro de masa" de la señal. Internamente, se calcula la FFT y se hace
# una media ponderada de las frecuencias de la señal con sus magnitudes como ponderaciones.
spec_centr = librosa.feature.spectral_centroid(y, sr)

print(spec_centr)

# Zero-crossing rate: indica las veces en las que la señal ha pasado por el valor 0 (ha pasado de positivo-negativo
# o de negativo-positivo)

zero_cross = librosa.feature.zero_crossing_rate(y)

print(zero_cross)

## Traditional Machine Learning

-Se extraen las características del audio (amplitude envelope, zero crossing rate...) y se intenta clasificar el audio en las clases que quieran (ruido coche, habla, pájaro...)

## Deep learning

-Se pasa el audio en crudo sin extraer ninguna característica 
-Se pueden pasar también características del audio y pasarlas por la red neuronal

### Features 
#### Time domain

- Amplitude envelope
- Root-mean square energy
- Zero crossing rate

#### Frequency domain (obtained by applying FFT)
- Band energy ratio
- Spectral centroid
- Spectral flux

#### Time-frequency representation
- Spectrogram (obtained by applying a STFT to time-domain signal)
- Mel-spectrogram
- Costant-Q transform